# POI Category by Population

With this notebook you can correlate any value associated with a geo-reference with the Google popularity score. You
can upload your own file as a CSV. The only thing that is necessary to make it work is to have columns for latitude and
longitude and column headers.

The value columns can be specific to your use case, e.g., scooter bookings, sales in shops or crimes. The popularity
score is aggregated on a week. So ideally, the value columns that you want to correlate are aggregated on a weekly
timeframe as well.

As an example we are using an open data set from Uber that gives us the traversals of rides through specific hexagons.
You can find the raw data on [their open data platform]('https://movement.uber.com/?lang=en-US). We preprocessed the raw
data so that the traversals are already aggregated per week.

## 1. Set Parameters

1. Set the file path to your CSV and the delimiter. Simply place your file under `kuwala/resources` from within the
Jupyter environment or under `kuwala/common/jupyter/resources` from the repository root on your local file system.

In [1]:
file_path = '../resources/lisbon_uber_traversals.csv'
delimiter = ';'

2. Set the H3 resolution to aggregate the results on.

    To see the average size of a hexagon at a given resolution go to the
    [official H3 documentation](https://h3geo.org/docs/core-library/restable). The currently set resolution 8 has on
    average an edge length of 0.46 km which can be freely interpreted as a radius.

In [2]:
resolution = 8

3. Set the column names for the coordinates and the columns of the file you want to correlate.

In [3]:
lat_column = 'latitude'
lng_column = 'longitude'
value_columns = ['weekly_traversals']

4. You can provide polygon coordinates as a GeoJSON-conform array to select a subregion. Otherwise, data form the entire
database will be analyzed. (The default coordinates are a rough representation of Lisbon, Portugal.)

In [4]:
polygon_coords = '[[[-9.092559814453125,38.794500078219826],[-9.164314270019531,38.793429729760994],[-9.217529296875,38.76666579487878],[-9.216842651367188,38.68792166352608],[-9.12139892578125,38.70399894245585],[-9.0911865234375,38.74551518488265],[-9.092559814453125,38.794500078219826]]]'


## 2. Load dataframes

#### Create a Spark session that is used to load your file and connect to the Neo4j instance .

In [5]:
from kuwala.modules.common import get_spark_session

sp = get_spark_session(memory_in_gb=16)

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
neo4j-contrib#neo4j-connector-apache-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-977ee0d0-3b59-45e9-ab25-7a06a7481bd3;1.0
	confs: [default]
	found neo4j-contrib#neo4j-connector-apache-spark_2.12;4.0.1_for_spark_3 in spark-packages
downloading https://repos.spark-packages.org/neo4j-contrib/neo4j-connector-apache-spark_2.12/4.0.1_for_spark_3/neo4j-connector-apache-spark_2.12-4.0.1_for_spark_3.jar ...
	[SUCCESSFUL ] neo4j-contrib#neo4j-connector-apache-spark_2.12;4.0.1_for_spark_3!neo4j-connector-apache-spark_2.12.jar (773ms)
:: resolution report :: resolve 4433ms :: artifacts dl 778ms
	:: modules in use:
	neo4j-contrib#neo4j-connector-apache-spark_2.12;4.0.1_for_spark_3 from spark-packages in [default]
	---------------------------------------------------------------------
	|                  |            modules            

#### Load the file

In [6]:
import json
from geojson import Polygon
from kuwala.modules.common import add_h3_index_column, polyfill_polygon

df_file = sp.read.option('delimiter', delimiter).csv(file_path, header=True)
df_file = add_h3_index_column(df=df_file, lat_column=lat_column, lng_column=lng_column, resolution=resolution)

if polygon_coords:
    polygon_coords_json = json.loads(polygon_coords)
    polygon = Polygon(polygon_coords_json)
    h3_index_in_polygon = list(polyfill_polygon(polygon=polygon, resolution=resolution))
    df_file = df_file.filter(df_file.h3_index.isin(h3_index_in_polygon))

aggregations = { x: 'sum' for x in value_columns}
df_file = df_file.select('h3_index', *value_columns).groupBy('h3_index').agg(aggregations)

df_file.show(n=10)

+---------------+----------------------+
|       h3_index|sum(weekly_traversals)|
+---------------+----------------------+
|8839336761fffff|              217524.0|
|8839336287fffff|               64503.0|
|88393375b9fffff|               17305.0|
|8839336281fffff|                2347.0|
|8839336709fffff|              168946.0|
|88393362b3fffff|             1520241.0|
|88393362d1fffff|              168001.0|
|883933629bfffff|              748052.0|
|88393375b3fffff|              329015.0|
|883933759bfffff|               33943.0|
+---------------+----------------------+
only showing top 10 rows



#### Get weekly popularity per hexagon

In [7]:
from kuwala.modules.popularity_controller import get_weekly_popularity_in_h3

popularity = get_weekly_popularity_in_h3(sp=sp, resolution=resolution, polygon_coords=polygon_coords)

popularity.show(n=10)

+---------------+-----------------+
|       h3_index|weekly_popularity|
+---------------+-----------------+
|8839336019fffff|            13801|
|8839336051fffff|            23673|
|8839336053fffff|            55269|
|8839336057fffff|            34827|
|8839336059fffff|            11658|
|883933605bfffff|            25050|
|88393360d9fffff|           109533|
|8839336213fffff|             6216|
|8839336281fffff|             4833|
|8839336283fffff|            36418|
+---------------+-----------------+
only showing top 10 rows



## 3. Join dataframes

In [8]:
popularity = popularity.withColumnRenamed('h3_index', 'join_h3_index')
result = df_file \
    .join(popularity, df_file.h3_index == popularity.join_h3_index, 'left') \
    .drop('join_h3_index') \
    .fillna(0, subset=['weekly_popularity'])

result.show(n=10)

+---------------+----------------------+-----------------+
|       h3_index|sum(weekly_traversals)|weekly_popularity|
+---------------+----------------------+-----------------+
|8839336761fffff|              217524.0|           120565|
|8839336287fffff|               64503.0|             8484|
|88393375b9fffff|               17305.0|            17450|
|8839336281fffff|                2347.0|             4833|
|8839336709fffff|              168946.0|             3820|
|88393362b3fffff|             1520241.0|           637042|
|88393362d1fffff|              168001.0|            47620|
|883933629bfffff|              748052.0|           133033|
|88393375b3fffff|              329015.0|            33861|
|883933759bfffff|               33943.0|            12501|
+---------------+----------------------+-----------------+
only showing top 10 rows



## 4. Visualize Results

#### Pandas Profiling Report

In [10]:
from pandas_profiling import ProfileReport

result_pd = result.toPandas()
profile = ProfileReport(result_pd, title="Pandas Profiling Report", explorative=True)

profile.to_widgets()

Summarize dataset:   0%|          | 0/16 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

#### Map

In [11]:
from unfolded.map_sdk import UnfoldedMap
from sidecar import Sidecar
from uuid import uuid4

unfolded_map = UnfoldedMap()
sc = Sidecar(title=f'Popularity Correlation', anchor='split-right')

with sc:
    display(unfolded_map)

dataset_id_combined=uuid4()

unfolded_map.add_dataset({
    'uuid': dataset_id_combined,
    'label': f'Correlated values',
    'data': result_pd
})

<Future pending>